## Imports

In [1]:
!echo $CUDA_VISIBLE_DEVICES

0,1


In [2]:
import torch
from torch import optim
from ignite.handlers import Checkpoint

In [3]:
%run -n train_classification.py

In [4]:
%run models/classification/__init__.py

In [5]:
%run datasets/__init__.py

In [6]:
%run models/checkpoint/__init__.py

In [7]:
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Load stuff

### Load datasets

In [8]:
dataset_name = 'cxr14'
kwargs = {
    'max_samples': None,
    'batch_size': 30,
}

train_dataloader = prepare_data_classification(dataset_name, 'train', **kwargs)
val_dataloader = prepare_data_classification(dataset_name, 'val', **kwargs)
train_dataloader.dataset.size()

Loading train dataset...
Loading val dataset...


(75714, 14)

### Load pretrained model

In [9]:
# run_name = '0704_005511_covid-kaggle_tfs-small_lr1e-06'
run_name = '0714_232500_cxr14_densenet-121_lr1e-06'
compiled_model = load_compiled_model_classification(run_name,
                                                    debug=False,
                                                    multiple_gpu=True,
                                                    device=DEVICE)

In [10]:
compiled_model.model

DataParallel(
  (module): Densenet121CNN(
    (base_cnn): DenseNet(
      (features): Sequential(
        (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
            (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer2): _Dens

### Create new model

In [6]:
lr = 0.000001
# run_name = f'cxr14_lr{lr}'
run_name = 'debugging'

model = init_empty_model('tfs-small',
                         train_dataloader.dataset.labels,
                         multilabel=train_dataloader.dataset.multilabel,
                         # pretrained_cnn=pretrained_cnn.base_cnn,
                         imagenet=True,
                         freeze=False,
                        ).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=lr)

compiled_model = CompiledModel(model, optimizer)

## Train

In [ ]:
%%capture output
%%time

print_metrics = ['loss', 'acc', 'hamming']
# print_metrics = ['loss', 'acc', 'spec_covid', 'recall_covid']

train_metrics, val_metrics = train_model(run_name,
                                         compiled_model,
                                         train_dataloader,
                                         val_dataloader,
                                         n_epochs=5,
                                         loss_name='focal',
                                         print_metrics=print_metrics,
                                         debug=False,
                                         device=DEVICE,
                                        )

In [15]:
print(output)

Resuming from epoch:  19
Using loss:  focal
--------------------------------------------------
Training...
Finished epoch 20/24 loss 7.9555 12.0304, acc 0.5365 0.3533, hamming 0.0505 0.0949, 0h 43m 53s
Finished epoch 21/24 loss 7.7393 12.0385, acc 0.5468 0.3567, hamming 0.0488 0.0940, 0h 44m 6s
Finished epoch 22/24 loss 7.5092 12.0463, acc 0.5578 0.3590, hamming 0.0471 0.0931, 0h 44m 9s
Finished epoch 23/24 loss 7.2867 12.0509, acc 0.5687 0.3622, hamming 0.0455 0.0923, 0h 44m 12s
Finished epoch 24/24 loss 7.0798 12.0894, acc 0.5812 0.3681, hamming 0.0437 0.0912, 0h 44m 14s
Average time per epoch:  0h 44m 14s
--------------------------------------------------
CPU times: user 16h 44min 34s, sys: 33min 40s, total: 17h 18min 15s
Wall time: 3h 40min 37s



## Debug stuff

### Test metrics

In [95]:
from ignite.metrics import Accuracy, Recall, Precision

In [143]:
%run ./metrics/classification/__init__.py
%run ./metrics/classification/specificity.py
%run ./metrics/classification/accuracy.py
%run ./metrics/classification/hamming.py

In [144]:
acc = MultilabelAccuracy(output_transform=_transform_remove_loss_and_round)
ham = Hamming(output_transform=_transform_remove_loss_and_round)

In [153]:
outputs = torch.tensor([[0, 1, 1],
                        [0.3, 0.7, 0.8],
                       ])
target = torch.tensor([[0, 0, 1],
                       [0, 1, 1],
                      ])

In [154]:
acc.reset()
acc.update(_transform_remove_loss_and_round((0, outputs, target)))
acc.compute()

0.6666666666666666

In [155]:
ham.reset()
ham.update(_transform_remove_loss_and_round((0, outputs, target)))
ham.compute()

0.16666666666666666

In [ ]:
sp = Specificity()
rec = Recall()
prec = Precision()

In [ ]:
fn = _get_transform_one_class(0)

In [ ]:
# outputs = torch.tensor([[1, 2, 1, 0, 0]])
# target = torch.tensor([[1, 0, 1, 1, 2]])
outputs = torch.tensor([[0, 20, -1],
                        [-40, 2, 3],
                        [17, 5, 6],
                       ])
target = torch.tensor([0, 0, 2])
outputs, target = fn((0, outputs, target))
outputs, target

In [ ]:
sp.reset()
sp.update((outputs, target))
sp.compute()

In [ ]:
rec.reset()
rec.update((outputs, target))
rec.compute().item()

In [ ]:
prec.reset()
prec.update((outputs, target))
prec.compute().item()

#### Test BCE loss

In [19]:
import torch
import numpy as np
from torch.nn.functional import binary_cross_entropy

In [41]:
%run losses/bce.py

In [45]:
EPS = 1e-5

In [86]:
target = torch.tensor([[1, 0, 0, 0, 0, 0],
                       [0, 0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 0, 0],
                      ])
bce = WeigthedBCELoss()

In [90]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(82.8836)

In [96]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(151.9611)

In [97]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(0.0004)

In [98]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(69.0779)

In [110]:
output = output_o.clamp(min=EPS, max=1-EPS)
output

tensor([[1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 9.9999e-01, 1.0000e-05, 9.9999e-01, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05]])

In [94]:
total = np.prod(target.size())
positive = (target == 1).sum().item()
negative = total - positive
total, positive, negative

(18, 3, 15)

In [95]:
BP = total / positive
BN = total / negative
BP, BN

(6.0, 1.2)

In [85]:
target.size(), output.size()

(torch.Size([2, 3]), torch.Size([3, 6]))

In [116]:
left = (target * torch.log(output))
left

tensor([[-1.1513e+01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -1.0014e-05, -0.0000e+00, -1.0014e-05,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00]])

In [117]:
right = (1-target) * torch.log(1-output)
right

tensor([[-0.0000e+00, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -0.0000e+00, -1.0014e-05, -0.0000e+00,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05]])

In [118]:
-(weights*(left + right)).sum()

tensor(69.0779)

In [99]:
weights = torch.zeros(target.size())
weights

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [109]:
weights[target == 0] = BN
weights[target == 1] = BP
weights

tensor([[6.0000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000],
        [1.2000, 1.2000, 6.0000, 1.2000, 6.0000, 1.2000],
        [1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000]])

In [119]:
binary_cross_entropy(output_o, target.float(), weight=weights, reduction='sum')

tensor(600.)

In [120]:
bce(output_o, target)

tensor(69.0779)

In [122]:
def calc_conv_output_size(input_size, padding, kernel_size, stride, dilation=0):
    value = (input_size + 2*padding - dilation * (kernel_size - 1) - 1)
    value /= (stride)
    value += 1
    return value

In [124]:
conv = nn.Conv2d(3, 32, (8, 8), stride=4)